In [ ]:
import tensorflow as tf
from six.moves import cPickle
import numpy as np
import time
import matplotlib
matplotlib.use('Agg')
from matplotlib import pyplot as plt
from PIL import Image
from model import *   # This has the model(neural network) we will be using for this problem
import pandas as pd

In [ ]:
def neural_net_image_input(image_shape):
        """
        Return a Tensor for a batch of image input
        : image_shape: Shape of the images
        : return: Tensor for image input.
        """
        return tf.placeholder(tf.float32,shape=(None,image_shape[0],image_shape[1],image_shape[2]),name="x")
def neural_net_label_input(image_shape):
        """
        Return a Tensor for a batch of image input
        : image_shape: Shape of the images
        : return: Tensor for image input.
        """
        return tf.placeholder(tf.uint8,shape=(None,image_shape[0],image_shape[1],image_shape[2]),name="y")

def neural_net_keep_prob_input():
    """
    Return a Tensor for keep probability
    : return: Tensor for keep probability.
    """
    return tf.placeholder(tf.float32,name="keep_prob")

In [ ]:
# We are using a model trained on pascal voc dataset, which has the following categories defined and finetuning the model for our purposes
label_colours = [(0,0,0)
                # 0=background
                ,(128,0,0),(0,128,0),(128,128,0),(0,0,128),(128,0,128)
                # 1=aeroplane, 2=bicycle, 3=bird, 4=boat, 5=bottle
                ,(0,128,128),(128,128,128),(64,0,0),(192,0,0),(64,128,0)
                # 6=bus, 7=car, 8=cat, 9=chair, 10=cow
                ,(192,128,0),(64,0,128),(192,0,128),(64,128,128),(192,128,128)
                # 11=diningtable, 12=dog, 13=horse, 14=motorbike, 15=person
                ,(0,64,0),(128,64,0),(0,192,0),(128,192,0),(0,64,128)]
                # 16=potted plant, 17=sheep, 18=sofa, 19=train, 20=tv/monitor
    
def decode_labels(mask):
    """Decode batch of segmentation masks.
    
    Args:
      label_batch: result of inference after taking argmax.
    
    Returns:
      An batch of RGB images of the same size
    """
    imgrgb = Image.new('RGB', (len(mask[0]), len(mask)))
    pixels = imgrgb.load()
    for j_, j in enumerate(mask):
        for k_, k in enumerate(j):
            if k < 21:
                pixels[k_,j_] = label_colours[k]
    return np.array(imgrgb)
def encode_labels(label_batch):
        colormap = {(0,0,0):0, (128,0,0):1, (0,128,0):2, (128,128,0):3, (0,0,128):4, (128,0,128):5, (0,128,128):6, (128,128,128):7, (64,0,0):8, (192,0,0):9, (64,128,0):10, (192,128,0):11, (64,0,128):12, (192,0,128):13, 
            (64,128,128):14, (192,128,128):15, (0,64,0):16, (128,64,0):17, (0,192,0):18, (128,192,0):19, (0,64,128):20}                                            
        gndTruth = np.zeros((label_batch.shape[0],label_batch[0].shape[0],label_batch[0].shape[1],1), dtype=np.int)
        for i in range(gndTruth.shape[0]):
            for j in range(gndTruth.shape[1]):
                for k in range(gndTruth.shape[2]):   
                    if(colormap.get(tuple(label_batch[i][j,k]))):
                        gndTruth[i,j,k]=colormap.get(tuple(label_batch[i][j,k]))
                    else:
                        gndTruth[i,j,k] = 0
        return gndTruth
def load_preprocess_training_batch(batch_id):
    """
    Load the Preprocessed Training data and return them in batches of <batch_size> or less
    """
    with tf.name_scope("create_inputs"):
        filename = 'preprocessed_data/train_data/pre_processed_batch_' + str(batch_id) + '.p'
        loaded_features, loaded_labels = cPickle.load(open(filename, mode='rb'))
        return loaded_features.astype(np.int64), loaded_labels.astype(np.int64)
    


In [ ]:
#CELEBA dataset has this file , which should be used to divide the dataset into 
#training - 0, 
#validation -1 
#testing - 2 

evalData = pd.read_csv('evalpartition.txt',sep=" ",header=None,skipinitialspace=True,names=['image_name','dataset_type'])
evalData.set_index('dataset_type',inplace=True)
trainData = evalData.loc[0].reset_index()['image_name']


In [ ]:
epochs = 50  # number of iterations
keep_probability = 0.75  # means 25% is dropout
save_model_path = "./finalModel"   # used to save model after each epoch
image_save_path = "images/"

im_sz = 500    # height and width to which all images will be resized to
batch_size=16  # dataset will be divided into batches of this size
num_train_batches = int(len(trainData)/batch_size)

In [1]:
# Inputs
x = neural_net_image_input((500, 500, 3))
y = neural_net_label_input((500, 500, 1))
keep_prob = neural_net_keep_prob_input()

# load model structure with loss and error functons defined from model.py
net = SegmentationModel(None)

saver = tf.train.Saver(max_to_keep=40)

# load model pretrained on PASCAL VOC augmented dataset 
def load_pretrained_model(sess):
    if not tf.train.checkpoint_exists("model/"):   # path to save checkpoints
        saver.restore(sess, "model/model.ckpt.data-00000-of-00001")
        print("initialised")
        

cost = net.loss(x, y,keep_prob)
cost_summary=tf.summary.scalar("loss",cost)
optim = tf.train.AdamOptimizer(learning_rate=1e-4).minimize(cost)
 
pred = net.preds(x)
merged = tf.summary.merge_all()

sv = tf.train.Supervisor(logdir="model", init_fn=load_pretrained_model, summary_op=None)

config = tf.ConfigProto()
config.gpu_options.allow_growth = True
with sv.managed_session(config=config) as sess:
    for epoch in range(epochs):
        for batch_i  in range(num_train_batches):
            if sv.should_stop():
                break
            start_time = time.time()
            batch_features, batch_labels = load_preprocess_training_batch(batch_i)
            loss_value,_=sess.run([cost,optim],feed_dict={x:batch_features,y:batch_labels,keep_prob:keep_probability})
            duration = time.time() - start_time
            print('step {:d}, batch {:d} \t loss = {:.3f}, ({:.3f} sec/step)'.format(epoch, batch_i, loss_value, duration)) 
        saver.save(sess, save_model_path)

NameError: name 'neural_net_image_input' is not defined